In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/01/28 16:11:24 WARN Utils: Your hostname, DESKTOP-8147UDP resolves to a loopback address: 127.0.1.1; using 172.18.36.72 instead (on interface eth0)
25/01/28 16:11:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 16:11:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
class invoiceStream():
    def __init__(self):
        self.base_data_dir = "/root/data_engineer2/Apache-Spark-and-Databricks-Stream-Processing-in-Lakehouse-main/my_practice/landing"

        self.json_schema = """InvoiceNumber string, CreatedTime bigint, StoreID string, PosID string, CashierID string,
                        CustomerType string, CustomerCardNo string, TotalAmount double, NumberOfItems bigint, 
                        PaymentMethod string, TaxableAmount double, CGST double, SGST double, CESS double, 
                        DeliveryType string,
                        DeliveryAddress struct<AddressLine string, City string, ContactNumber string, PinCode string, 
                        State string>,
                        InvoiceLineItems array<struct<ItemCode string, ItemDescription string, 
                            ItemPrice double, ItemQty bigint, TotalValue double>>
                    """
    
    def readInvoices(self):
        return (spark.readStream
                .format("json")  
                .schema(self.json_schema)   
                .load(f"{self.base_data_dir}/data/invoices")) 
    
    def explodeInvoices(self, invoiceDF):
        return ( invoiceDF.selectExpr("InvoiceNumber", "CreatedTime", "StoreID", "PosID",
                                      "CustomerType", "PaymentMethod", "DeliveryType", "DeliveryAddress.City",
                                      "DeliveryAddress.State","DeliveryAddress.PinCode", 
                                      "explode(InvoiceLineItems) as LineItem")
                                    )  

    def flattenInvoices(self, explodedDF):
        return( explodedDF.withColumn("ItemCode", expr("LineItem.ItemCode"))
                        .withColumn("ItemDescription", expr("LineItem.ItemDescription"))
                        .withColumn("ItemPrice", expr("LineItem.ItemPrice"))
                        .withColumn("ItemQty", expr("LineItem.ItemQty"))
                        .withColumn("TotalValue", expr("LineItem.TotalValue"))
                        .drop("LineItem")
                )

    def appendInvoices(self, flattenedDF: DataFrame):
        return (flattenedDF.writeStream
                    .format("parquet")
                    .option("checkpointLocation", f"{self.base_data_dir}/chekpoint/invoices")
                    .outputMode("append")
                    .start("/root/data_engineer2/Apache-Spark-and-Databricks-Stream-Processing-in-Lakehouse-main/my_practice/output/inv-stream")
        )
    
    def process(self):
        print(f"Starting Invoice Processing Stream...", end='')
        invoicesDF = self.readInvoices()
        explodedDF = self.explodeInvoices(invoicesDF)
        resultDF = self.flattenInvoices(explodedDF)
        sQuery = self.appendInvoices(resultDF)
        print("Done\n")
        return sQuery     

NameError: name 'DataFrame' is not defined

In [15]:
inv_stream = invoiceStream()
sQuery = inv_stream.process()
sQuery.explain()

Starting Invoice Processing Stream...Done

No physical plan. Waiting for data.


24/12/04 10:43:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/04 10:43:45 WARN StreamingQueryManager: Stopping existing streaming query [id=07c62567-0f82-4b53-a416-c9969382a5ce, runId=c415efb1-5289-4b2f-82f2-4e7d0cd9ddad], as a new run is being started.


In [13]:
sQuery.stop()